In [1]:
import numpy as np
import pandas as pd
# pd.set_option('float_format', '{:f}'.format)
import matplotlib.pyplot as plt

from sympy.combinatorics import Permutation
from sympy import init_printing
init_printing(perm_cyclic=False, pretty_print=False)
from copy import deepcopy as dc

from collections import namedtuple
from tqdm import tqdm
import time

# методы из прошлой лабы

In [2]:
def norm(x ):
    return np.sqrt(sum([xi**2 for xi in x]))

def scalar_product(x,y):
    return (sum([xi*yi for xi, yi in zip(x,y)]))

def mat_vec_product(a, x):
    assert len(a[0]) == len(x)

    return np.array([
        scalar_product(y,x)
        for y in a
    ])

def matrix_product(a, b):
    k, l = len(a), len(a[0])
    l2, m = len(b), len(b[0])
    assert l == l2

    return np.array(
        [
            [
                sum(a[i][t]*b[t][j] for t in range(l))
                for j in range(m)
            ]
            for i in range(k)
        ]
    )

def transpose(a):
    return np.array([
        [a[j][i]for j in range(len(a))]
        for i in range(len(a[0]))
    ])



In [3]:
def plotter(f, a, b, dots_num=100):
    x = np.linspace(a,b, dots_num, True)
    y = f(x)
    plt.plot(x,y)
    plt.show()

# метод гаусса

In [23]:
NORMAL = 1
SWAP_ROWS = 2
SWAP_COLS = 3
SWAP_ALL = 4

def max_elem_in_matrix(A, iter_num):
    m = 0
    mi, mj = -1,-1
    for i in range(iter_num, len(A)):
        for j in range(iter_num, len(A)):
            if abs(A[i][j]) > m:
                m = abs(A[i][j])
                mi, mj = i,j
    return mi, mj

def max_elem_in_row(A, iter_num):
    m = 0
    mi, mj = -1,-1
    i = iter_num
    for j in range(iter_num, len(A)):
        if abs(A[i][j]) > m:
            m = abs(A[i][j])
            mi, mj = i,j

    return mi, mj


def max_elem_in_column(A, iter_num):
    m = 0
    mi, mj = -1,-1
    j = iter_num
    for i in range(iter_num, len(A)):
        if abs(A[i][j]) > m:
            m = abs(A[i][j])
            mi, mj = i,j
    return mi, mj

def permute_rows(A, f, i, j):
    for t in range(len(A)):
        A[i][t], A[j][t] = A[j][t], A[i][t]
    f[i], f[j] = f[j], f[i]
    return A, f

def permute_cols(A, f, i, j):
    for t in range(len(A)):
        A[t][i], A[t][j] = A[t][j], A[t][i]
    return A, f

def gaussian(A, f, mode=NORMAL):
    assert len(A) == len(A[0]) == len(f)
    A, f = dc(A), dc(f)
    D = np.zeros((len(A), len(A)))
    Dr = np.zeros((len(A),))
    N = len(A)
    p = Permutation()

    # прямой ход
    for iter_num in range(N-1):
        for t in range(iter_num+1, N):
            c = A[t, iter_num]/A[iter_num, iter_num]
            A[t] = A[t] - c*A[iter_num]
            D[t] = abs(c) * abs(A[iter_num]) + abs(A[t])
            f[t] = f[t] - c*f[iter_num]
            Dr[t] = abs(f[t]) + abs(c) + abs(f[iter_num])

    # обратный ход
    x = np.zeros(N)
    for iter_num in range(N-1, -1, -1):
        x[iter_num] = (f[iter_num] - scalar_product(x, A[iter_num])) / A[iter_num, iter_num]

    # перестановка элементов x
    for i,j in p.transpositions():
        i,j = i-1, j-1
        x[i], x[j] = x[j], x[i]

    print(D)
    print(Dr)
    print('delta norm:', max(np.linalg.norm(D), np.linalg.norm(Dr)))


    return x

## проверка

In [24]:
A = np.random.rand(5,5)
f = np.random.rand(5)

In [25]:
x = gaussian(A, f)

print(A@x)
print(f)

[[0.         0.         0.         0.         0.        ]
 [0.25764382 0.6855051  0.25025911 0.95562458 0.28177488]
 [0.         0.0671283  0.89234173 0.49249202 0.04093266]
 [0.         0.         0.28085231 0.48483122 0.09753412]
 [0.         0.         0.         0.25905381 0.28700987]]
[0.         1.30280296 0.89370698 1.11226048 1.63229611]
delta norm: 2.5293322622916294
[0.87822307 0.38618883 0.73437477 0.85749108 0.52684888]
[0.87822307 0.38618883 0.73437477 0.85749108 0.52684888]


# метод томаса

# создание матриц

In [69]:
def input_matrix():
    a, b = int(input()), int(input())
    A = np.zeros((a,b))
    for i in range(a):
        for j in range(b):
            A[i,j] = float(input())

    return A

In [70]:
def gen_matrix(n,m,a=-1, b=1, dd=None):
    A = np.random.rand(n,m)
    A *= b-a
    A += a

    if dd is None:
        return A

    if n != m:
        raise RuntimeError(f'диагональное преобладание возможно только у квадратных матриц')

    # генерируем коэффициенты для диагонального преобладания
    dd_coefs = [dd * np.random.random() for _ in range(n)]
    dd_coefs[np.random.randint(0, n)] = 1 * dd
    for i in range(n):
        A[i,i] = (sum(abs(A[i])) - abs(A[i,i])) + dd_coefs[i]


    return A

In [71]:
from pprint import pp

In [72]:
pp(gen_matrix(4,4, dd=10))

array([[ 1.05700156e+01, -8.92616640e-01, -9.45386076e-01,
        -3.30569812e-02],
       [-6.55904800e-01,  1.22361289e+01,  9.78919880e-01,
        -6.01304190e-01],
       [-6.13078060e-01,  4.01048899e-02,  6.13395892e+00,
        -3.01068310e-01],
       [ 1.21672206e-01,  7.78550769e-03,  6.86191380e-01,
         5.00117886e+00]])


# тестирование

## функции тестирования

In [73]:
HyperParams = namedtuple('HyperParams', ['diag_dom', 'N'])
Test = namedtuple('Test', ['name', 'func'])

In [79]:
def run_tests(hyperparams, tests):
    '''
    вычисляет значения для всех тестов и наборов гиперпараметров
    '''

    # создаем табличку для результатов
    res_df = pd.DataFrame(columns = ['method', 'res', 'N', 'diag_dom', 'x_true', 'time'])

    # заполняем табличку
    for hp in tqdm(hyperparams, position=0, leave=True):
        # создаем тестовые данные
        A = gen_matrix(hp.N,hp.N, -1,1, hp.diag_dom)
        x = np.random.rand(hp.N)
        b = mat_vec_product(A, x)
        for test in tests:
            # вычисляем и сохраняем
            t = time.time()
            res_x = test.func(A,b)
            t2 = time.time()
            res_df.loc[len(res_df)] = [test.name, res_x, hp.N, hp.diag_dom, x, t2-t]

    return res_df


def process_results(res_df):
    res_df['abs_diff'] = (res_df['res'] - res_df['x_true']).map(norm)
    res_df = res_df.drop(['res', 'x_true'], axis=1)
    return res_df


def draw_results(res_df):
    methods = res_df.groupby('N')
    for N, ngroup in res_df.groupby('N'):
        plt.figure()
        for name, method_df in ngroup.groupby('method'):
            xls = np.arange(len(method_df.diag_dom))
            plt.plot(xls, method_df.abs_diff, 'o--', label=name)
            plt.xticks(np.arange(len(method_df.diag_dom)), map(lambda x:"{:3.2f}".format(x) if isinstance(x, int) else 'none', method_df.diag_dom))
        plt.title(f'N = {N}')
        plt.legend()
        plt.grid()
        plt.show()

In [80]:
def combo(hps, tsts):
    res_df = run_tests(hps, tsts)
    res_df = process_results(res_df)
    print(res_df)
    # draw_results(res_df)

## определим тесты

In [85]:
HP_LIST_BIG = []
for N in [100]:
    for dd in [150]:
        HP_LIST_BIG.append(HyperParams(dd,N))

HP_LIST_SMALL = []
for N in [100]:
    for dd in [-25, -10, 0, 5, 10]:
        HP_LIST_SMALL.append(HyperParams(dd,N))

TESTS = [
    Test('normal', lambda x,y: gaussian(x,y, NORMAL)),
    Test('swap_rows', lambda x,y: gaussian(x,y, SWAP_ROWS)),
    Test('swap_cols', lambda x,y: gaussian(x,y, SWAP_COLS)),
    Test('swap_all', lambda x,y: gaussian(x,y, SWAP_ALL)),
]

## запускаем тесты

In [86]:
combo(HP_LIST_BIG, TESTS)

100%|██████████| 1/1 [00:00<00:00,  2.14it/s]

      method    N  diag_dom      time      abs_diff
0     normal  100       150  0.040716  3.666567e-15
1  swap_rows  100       150  0.057816  3.666567e-15
2  swap_cols  100       150  0.072892  3.666567e-15
3   swap_all  100       150  0.274651  3.810604e-15
